In [28]:
import pandas as pd

from xgboost import XGBRegressor

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error, r2_score

import matplotlib.pyplot as plt

In [29]:
F_selection = pd.read_csv("F_Selection/top_N_features_SelectFromModel_XGBAll.csv")

# Extract top 10, 20, 30, and 50 features
top_10 = F_selection["Feature"].head(10).tolist()
top_20 = F_selection["Feature"].head(20).tolist()
top_30 = F_selection["Feature"].head(30).tolist()
top_40 = F_selection["Feature"].head(40).tolist()
top_50 = F_selection["Feature"].head(50).tolist()

In [30]:
def Data_Organizer(raw_Data,top):
    
    All_players_dataSet = raw_Data.copy()
    
    All_players_dataSet = All_players_dataSet.drop(columns=All_players_dataSet.columns[All_players_dataSet.columns.str.contains('Unnamed:')])
    All_players_dataSet.reset_index(drop=True, inplace=True)
    
    All_players_dataSet.fillna(0, inplace=True)
    All_players_dataSet.replace('', 0, inplace=True)
    All_players_dataSet.replace('--', 0, inplace=True)
    
    # check if any NaN,empty Strings exists in the dataframe
    any_missing_values = All_players_dataSet.isna().any().any()
    any_empty_values = (All_players_dataSet.applymap(lambda x: x == '')).any().any()
    
    if any_missing_values or any_empty_values:
        print("DataFrame contains missing values or empty strings/spaces.")
    else:
        print("DataFrame does not contain missing values or empty strings/spaces.")
        
    ## seperating our dataFrame
    Y = All_players_dataSet['Points_won']  # Target
    X = All_players_dataSet.drop('Points_won', axis=1)  # Features
    
    X = X.apply(pd.to_numeric, errors='coerce')

    ## keep the specified columns
    columns_to_keep = top
    
    ## keep only wanted columns
    X = X[columns_to_keep]

    return X, Y

In [31]:
all_players_w_add_sorted = pd.read_csv("all_players_w_add_sorted.csv")

C:\Users\Abdou\AppData\Local\Temp\ipykernel_13908\1174582640.py:1: DtypeWarning: Columns (154,155) have mixed types. Specify dtype option on import or set low_memory=False.
  all_players_w_add_sorted = pd.read_csv("all_players_w_add_sorted.csv")


In [32]:
top = top_10

rank_data_10 = pd.DataFrame()

for year in range(1994,2024):
    print("Selected Features Top ",10," Test year : ", year)
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] < year]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a XGBoost Regressor model
    xgb_model = XGBRegressor(n_estimators=200,max_depth = 5, random_state=42, n_jobs=-1)
    xgb_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_xgb = xgb_model.predict(X_test)
    
    # Evaluate the model
    mse_xgb = mean_squared_error(y_test, y_pred_xgb)
    r2_xgb = r2_score(y_test, y_pred_xgb)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (XGBoost): {mse_xgb}')
    print(f'R-squared (XGBoost): {r2_xgb}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_xgb
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)
    
    if rank_data_10.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_10 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_10 = pd.concat([rank_data_10.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_10)

Selected Features Top  10  Test year :  1994
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 7.188681032209482
R-squared (XGBoost): -0.9461744891868107


,Player_name,Rank,Pos,Points_won,Predicted_points
15651,Hakeem Olajuwon,1,C,23.0,72.997528
15477,Scottie Pippen,4,SF,11.0,8.936706
15583,Mookie Blaylock,7T,PG,3.0,8.917365
15539,David Robinson,2,C,22.0,8.903131
15435,Otis Thorpe,-1.0,PF,0.0,1.894845
15531,LaPhonso Ellis,-1.0,PF,0.0,0.996227
15784,Gary Payton,5,PG,9.0,0.879920
15743,Dikembe Mutombo,3,C,19.0,0.767310
15739,Patrick Ewing,-1,C,0.0,0.651609
15649,Charles Oakley,9,PF,2.0,0.336138


---------------------------------------------------------------------
Selected Features Top  10  Test year :  1995
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 3.3158190297360655
R-squared (XGBoost): 0.4729714575298841


,Player_name,Rank,Pos,Points_won,Predicted_points
15026,Dikembe Mutombo,1,C,45.0,72.979698
14994,Hakeem Olajuwon,3,C,13.0,22.994846
15199,Scottie Pippen,2,SF,16.0,21.997505
15051,Patrick Ewing,-1,C,0.0,18.721085
15113,David Robinson,4,C,12.0,13.751002
15077,Mookie Blaylock,-1,PG,0.0,2.693089
15394,Karl Malone,-1.0,PF,0.0,2.044275
15174,Vlade Divac,-1.0,C,0.0,1.938754
15241,Rod Strickland,-1.0,PG,0.0,0.742767
15374,Toni Kukoč,-1.0,SF,0.0,0.569358


---------------------------------------------------------------------
Selected Features Top  10  Test year :  1996
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 3.1873954784472835
R-squared (XGBoost): 0.5751746604110433


...

,Player_name,Rank,Pos,Points_won,Predicted_points
1022,Rudy Gobert,3.0,C,136.0,217.645737
1111,Giannis Antetokounmpo,6.0,PF,58.0,136.507507
713,Marcus Smart,1,PG,257.0,68.831451
1212,Jayson Tatum,-1.0,SF,0.0,46.672813
933,Al Horford,9,C,3.0,37.560493
1184,Nikola Jokić,-1.0,C,0.0,22.661442
1007,Joel Embiid,-1,C,0.0,22.571201
997,Jaren Jackson Jr.,5.0,PF,99.0,18.671509
1306,Bam Adebayo,4.0,C,128.0,18.360332
1026,Myles Turner,-1.0,C,0.0,15.771612


---------------------------------------------------------------------
Selected Features Top  10  Test year :  2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 314.23383627279713
R-squared (XGBoost): 0.2615248797244031


,Player_name,Rank,Pos,Points_won,Predicted_points
59,Jaren Jackson Jr.,1.0,C,391.0,91.542976
176,Giannis Antetokounmpo,6.0,PF,14.0,67.674706
208,Walker Kessler,-1.0,C,0.0,20.923491
323,Brook Lopez,2.0,C,309.0,16.097940
301,Anthony Davis,-1,C,0.0,13.355062
392,Jacob Gilyard,-1.0,PG,0.0,11.342652
206,Nic Claxton,9T,C,7.0,9.976711
60,Alex Caruso,11.0,PG,2.0,9.858918
497,Joel Embiid,9T,C,7.0,6.710688
309,Matisse Thybulle,-1,SG,0.0,6.463059


---------------------------------------------------------------------


,Rank_1994,Rank_1995,Rank_1996,Rank_1997,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,1,1,1,2,1,2,1,-1,3T,1,...,1.0,3.0,1,1,-1.0,1.0,3.0,1.0,3.0,1.0
1,4,3,3,1,2,1,3T,-1,-1,-1,...,4.0,10T,3,2.0,1.0,2.0,11,9.0,6.0,6.0
2,7T,2,5,-1,-1,3,5T,1,-1,4,...,-1.0,4,2,-1,15T,5.0,8T,5.0,1,-1.0
3,2,-1,4,4,11T,7T,-1,1,1,-1,...,15T,1,4.0,3,-1.0,-1,-1.0,3,-1.0,2.0
4,-1.0,4,-1.0,-1.0,3,4,2,5T,-1,-1.0,...,-1,8,10,5T,-1,3.0,2,6.0,9,-1
5,-1.0,-1,-1,3,-1,-1.0,-1,5T,-1.0,3,...,8T,-1.0,7.0,-1,-1.0,-1,10.0,-1,-1.0,-1.0
6,5,-1.0,6,-1.0,5T,5T,-1,3,-1.0,-1.0,...,5,15T,-1.0,-1,8.0,-1,1.0,11T,-1,9T
7,3,-1.0,-1.0,-1.0,4,-1.0,-1.0,-1,-1,-1.0,...,3.0,-1.0,5.0,-1,-1.0,6T,6T,10,5.0,11.0
8,-1,-1.0,2,6T,-1.0,-1,7,-1.0,-1,-1.0,...,7.0,-1.0,-1,-1.0,-1.0,6T,-1.0,-1.0,4.0,9T
9,9,-1.0,-1.0,-1.0,-1.0,-1.0,5T,11T,2,-1.0,...,-1.0,-1,-1.0,-1.0,7,8,-1.0,-1.0,-1.0,-1


In [33]:
top = top_20

rank_data_20 = pd.DataFrame()

for year in range(1994,2024):
    print("Selected Features Top ",20," Test year : ", year)
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] < year]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a XGBoost Regressor model
    xgb_model = XGBRegressor(n_estimators=200,max_depth = 5, random_state=42, n_jobs=-1)
    xgb_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_xgb = xgb_model.predict(X_test)
    
    # Evaluate the model
    mse_xgb = mean_squared_error(y_test, y_pred_xgb)
    r2_xgb = r2_score(y_test, y_pred_xgb)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (XGBoost): {mse_xgb}')
    print(f'R-squared (XGBoost): {r2_xgb}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_xgb
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)
    
    if rank_data_20.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_20 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_20 = pd.concat([rank_data_20.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_20)

Selected Features Top  20  Test year :  1994
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 7.21713373511267
R-squared (XGBoost): -0.9538774216566877


,Player_name,Rank,Pos,Points_won,Predicted_points
15651,Hakeem Olajuwon,1,C,23.0,72.998779
15477,Scottie Pippen,4,SF,11.0,8.881651
15583,Mookie Blaylock,7T,PG,3.0,8.855267
15539,David Robinson,2,C,22.0,8.813155
15739,Patrick Ewing,-1,C,0.0,1.845985
15435,Otis Thorpe,-1.0,PF,0.0,1.653562
15531,LaPhonso Ellis,-1.0,PF,0.0,1.605274
15577,Shawn Kemp,-1.0,PF,0.0,1.466537
15613,Karl Malone,-1.0,PF,0.0,1.128452
15525,Dennis Rodman,6,PF,8.0,1.008812


---------------------------------------------------------------------
Selected Features Top  20  Test year :  1995
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 2.856330144337982
R-squared (XGBoost): 0.5460043207171872


,Player_name,Rank,Pos,Points_won,Predicted_points
15026,Dikembe Mutombo,1,C,45.0,72.972687
14994,Hakeem Olajuwon,3,C,13.0,23.003784
15113,David Robinson,4,C,12.0,21.294519
15199,Scottie Pippen,2,SF,16.0,20.804983
15077,Mookie Blaylock,-1,PG,0.0,6.164573
14993,Tyrone Hill,-1.0,C,0.0,3.644063
15397,Dale Davis,-1.0,PF,0.0,3.573939
15329,Grant Long,-1.0,PF,0.0,3.335885
15404,Alonzo Mourning,6T,C,2.0,3.296584
15051,Patrick Ewing,-1,C,0.0,3.201173


---------------------------------------------------------------------
Selected Features Top  20  Test year :  1996
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 3.6284702669550186
R-squared (XGBoost): 0.5163869297767492


...

,Player_name,Rank,Pos,Points_won,Predicted_points
1022,Rudy Gobert,3.0,C,136.0,123.172592
1111,Giannis Antetokounmpo,6.0,PF,58.0,92.756042
1007,Joel Embiid,-1,C,0.0,36.331364
713,Marcus Smart,1,PG,257.0,29.923325
1184,Nikola Jokić,-1.0,C,0.0,28.256666
1123,Draymond Green,10,PF,2.0,28.203093
1012,Xavier Tillman Sr.,-1.0,PF,0.0,20.822206
628,Mikal Bridges,2.0,SF,202.0,20.605846
1306,Bam Adebayo,4.0,C,128.0,15.332803
1176,Jae Crowder,-1,PF,0.0,10.387307


---------------------------------------------------------------------
Selected Features Top  20  Test year :  2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 360.08584056695105
R-squared (XGBoost): 0.1537689334277511


,Player_name,Rank,Pos,Points_won,Predicted_points
206,Nic Claxton,9T,C,7.0,102.601387
59,Jaren Jackson Jr.,1.0,C,391.0,56.931034
176,Giannis Antetokounmpo,6.0,PF,14.0,51.069271
497,Joel Embiid,9T,C,7.0,25.489862
477,Bruno Fernando,-1.0,C,0.0,24.999241
323,Brook Lopez,2.0,C,309.0,19.323839
536,Andre Drummond,-1,C,0.0,18.040567
524,Bam Adebayo,5.0,C,18.0,14.789799
15,Domantas Sabonis,-1.0,C,0.0,8.869967
208,Walker Kessler,-1.0,C,0.0,8.140217


---------------------------------------------------------------------


,Rank_1994,Rank_1995,Rank_1996,Rank_1997,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,1,1,3,2,1,1,1,-1,1,1,...,1.0,1,1,1,1.0,1.0,1.0,1.0,3.0,9T
1,4,3,1,1,2,3,5T,3,3T,4,...,15T,3.0,3,3,6,2.0,11,9.0,6.0,1.0
2,7T,4,4,-1,3,2,2,1,-1,-1.0,...,7.0,4,2,2.0,15T,5.0,-1.0,5.0,-1,6.0
3,2,2,5,4,5T,5T,-1,1,-1,-1,...,11,2,4.0,-1,-1.0,6T,10.0,-1.0,1,9T
4,-1,-1,-1.0,-1.0,-1,-1.0,-1,11T,2,-1.0,...,4.0,-1,-1.0,-1.0,8.0,-1,3.0,-1,-1.0,-1.0
5,-1.0,-1.0,-1.0,3,4,7T,7,5T,-1.0,11,...,3.0,7.0,10,5T,-1.0,-1.0,2,3,10,2.0
6,-1.0,-1.0,2,5,-1.0,7T,-1.0,-1,-1,-1.0,...,2.0,10T,-1.0,-1,14.0,-1,-1.0,11T,-1.0,-1
7,-1.0,-1.0,-1.0,-1.0,11T,-1.0,3T,5T,-1.0,3,...,13T,12.0,-1.0,-1.0,3,4,-1.0,-1.0,2.0,5.0
8,-1.0,6T,6,-1.0,5T,-1.0,-1,-1.0,-1.0,-1.0,...,-1.0,8,-1,-1.0,-1.0,-1,-1.0,-1.0,4.0,-1.0
9,6,-1,-1.0,-1.0,-1.0,4,3T,4.0,-1,-1.0,...,8T,-1.0,5.0,-1.0,11.0,-1.0,-1,10,-1,-1.0


In [34]:
top = top_30

rank_data_30 = pd.DataFrame()

for year in range(1994,2024):
    print("Selected Features Top ",30," Test year : ", year)
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] < year]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a XGBoost Regressor model
    xgb_model = XGBRegressor(n_estimators=200,max_depth = 5, random_state=42, n_jobs=-1)
    xgb_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_xgb = xgb_model.predict(X_test)
    
    # Evaluate the model
    mse_xgb = mean_squared_error(y_test, y_pred_xgb)
    r2_xgb = r2_score(y_test, y_pred_xgb)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (XGBoost): {mse_xgb}')
    print(f'R-squared (XGBoost): {r2_xgb}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_xgb
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)

    if rank_data_30.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_30 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_30 = pd.concat([rank_data_30.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_30)

Selected Features Top  30  Test year :  1994
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 7.10241735986403
R-squared (XGBoost): -0.9228205306914945


,Player_name,Rank,Pos,Points_won,Predicted_points
15651,Hakeem Olajuwon,1,C,23.0,72.998627
15477,Scottie Pippen,4,SF,11.0,8.999494
15583,Mookie Blaylock,7T,PG,3.0,7.869447
15539,David Robinson,2,C,22.0,7.869447
15784,Gary Payton,5,PG,9.0,2.130140
15677,Nate McMillan,7T,PG,3.0,2.130140
15540,Horace Grant,10,PF,1.0,2.130140
15649,Charles Oakley,9,PF,2.0,1.000093
15739,Patrick Ewing,-1,C,0.0,1.000093
15525,Dennis Rodman,6,PF,8.0,1.000093


---------------------------------------------------------------------
Selected Features Top  30  Test year :  1995
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 3.171308556836127
R-squared (XGBoost): 0.495940486666008


,Player_name,Rank,Pos,Points_won,Predicted_points
15026,Dikembe Mutombo,1,C,45.0,72.990662
14994,Hakeem Olajuwon,3,C,13.0,22.999355
15113,David Robinson,4,C,12.0,22.002148
15199,Scottie Pippen,2,SF,16.0,14.920372
15404,Alonzo Mourning,6T,C,2.0,12.607483
15151,Gary Payton,6T,PG,2.0,12.116823
15077,Mookie Blaylock,-1,PG,0.0,7.242426
15358,Dan Majerle,-1,SF,0.0,5.050086
15121,Dennis Rodman,5,PF,11.0,4.793456
15249,Nate McMillan,6T,PG,2.0,4.432107


---------------------------------------------------------------------
Selected Features Top  30  Test year :  1996
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 3.541362838243166
R-squared (XGBoost): 0.527996861218724


...

,Player_name,Rank,Pos,Points_won,Predicted_points
1022,Rudy Gobert,3.0,C,136.0,184.758713
1111,Giannis Antetokounmpo,6.0,PF,58.0,116.179085
992,Paul George,-1.0,PF,0.0,52.406456
1123,Draymond Green,10,PF,2.0,45.178772
997,Jaren Jackson Jr.,5.0,PF,99.0,30.417896
1046,Sekou Doumbouya,-1.0,PF,0.0,24.754395
933,Al Horford,9,C,3.0,24.074516
813,Ricky Rubio,-1.0,PG,0.0,19.936605
713,Marcus Smart,1,PG,257.0,17.873274
1007,Joel Embiid,-1,C,0.0,16.779726


---------------------------------------------------------------------
Selected Features Top  30  Test year :  2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 383.0918843309493
R-squared (XGBoost): 0.09970285595754769


,Player_name,Rank,Pos,Points_won,Predicted_points
176,Giannis Antetokounmpo,6.0,PF,14.0,115.114929
137,Evan Mobley,3.0,PF,101.0,58.340946
497,Joel Embiid,9T,C,7.0,46.668465
408,Jarrett Allen,-1,C,0.0,45.239479
59,Jaren Jackson Jr.,1.0,C,391.0,42.116253
206,Nic Claxton,9T,C,7.0,30.482992
392,Jacob Gilyard,-1.0,PG,0.0,26.356594
524,Bam Adebayo,5.0,C,18.0,19.468533
75,Myles Turner,-1.0,C,0.0,17.361799
208,Walker Kessler,-1.0,C,0.0,15.592331


---------------------------------------------------------------------


,Rank_1994,Rank_1995,Rank_1996,Rank_1997,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,1,1,3,2,2,1,1,-1,1,1,...,1.0,1,1,1,1.0,2.0,1.0,1.0,3.0,6.0
1,4,3,1,1,1,3,5T,3,3T,8,...,15T,3.0,3,3,6,1.0,3.0,9.0,6.0,3.0
2,7T,4,4,-1,5T,5T,8T,1,-1,5.0,...,4.0,-1,2,2.0,-1,6T,2,-1.0,-1.0,9T
3,2,2,5,4,3,2,2,1,-1,4,...,7.0,4,4.0,5T,8.0,-1.0,10.0,-1,10,-1
4,5,6T,7,5,5T,4,8T,-1,3T,-1,...,3.0,2,5.0,-1.0,3,3.0,-1.0,5.0,5.0,1.0
5,7T,6T,6,6T,5T,7T,-1,2,-1.0,-1.0,...,11,10T,10,-1,-1,5.0,-1.0,-1,-1.0,9T
6,10,-1,2,3,4,7T,3T,5T,-1,11,...,2.0,7.0,-1.0,4.0,-1,6T,11,11T,9,-1.0
7,9,-1,-1,-1,11T,7T,-1,4.0,-1,13T,...,8T,15T,-1.0,-1,-1,4,-1.0,-1,-1.0,5.0
8,-1,5,-1,-1,11T,-1,-1,11T,-1,3,...,8T,-1,11T,-1.0,-1.0,-1.0,-1,2.0,1,-1.0
9,6,6T,-1,-1,9T,7T,7,-1,2,-1.0,...,5,8,-1,-1.0,15T,-1,-1,-1.0,-1,-1.0


In [35]:
top = top_40

rank_data_40 = pd.DataFrame()

for year in range(1994,2024):
    print("Selected Features Top ",40," Test year : ", year)
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] < year]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a XGBoost Regressor model
    xgb_model = XGBRegressor(n_estimators=200,max_depth = 5, random_state=42, n_jobs=-1)
    xgb_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_xgb = xgb_model.predict(X_test)
    
    # Evaluate the model
    mse_xgb = mean_squared_error(y_test, y_pred_xgb)
    r2_xgb = r2_score(y_test, y_pred_xgb)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (XGBoost): {mse_xgb}')
    print(f'R-squared (XGBoost): {r2_xgb}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_xgb
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)

    if rank_data_40.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_40 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_40 = pd.concat([rank_data_40.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_40)

Selected Features Top  40  Test year :  1994
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 7.10241735986403
R-squared (XGBoost): -0.9228205306914945


,Player_name,Rank,Pos,Points_won,Predicted_points
15651,Hakeem Olajuwon,1,C,23.0,72.998627
15477,Scottie Pippen,4,SF,11.0,8.999494
15583,Mookie Blaylock,7T,PG,3.0,7.869447
15539,David Robinson,2,C,22.0,7.869447
15784,Gary Payton,5,PG,9.0,2.130140
15677,Nate McMillan,7T,PG,3.0,2.130140
15540,Horace Grant,10,PF,1.0,2.130140
15649,Charles Oakley,9,PF,2.0,1.000093
15739,Patrick Ewing,-1,C,0.0,1.000093
15525,Dennis Rodman,6,PF,8.0,1.000093


---------------------------------------------------------------------
Selected Features Top  40  Test year :  1995
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 2.8770197103769193
R-squared (XGBoost): 0.5427158445560776


,Player_name,Rank,Pos,Points_won,Predicted_points
15113,David Robinson,4,C,12.0,21.554382
15026,Dikembe Mutombo,1,C,45.0,17.928368
14994,Hakeem Olajuwon,3,C,13.0,17.242222
15199,Scottie Pippen,2,SF,16.0,15.063243
15151,Gary Payton,6T,PG,2.0,13.451194
15404,Alonzo Mourning,6T,C,2.0,9.098365
15077,Mookie Blaylock,-1,PG,0.0,7.746881
15249,Nate McMillan,6T,PG,2.0,7.128888
15121,Dennis Rodman,5,PF,11.0,6.092377
15258,Derrick McKey,6T,SF,2.0,5.814053


---------------------------------------------------------------------
Selected Features Top  40  Test year :  1996
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 9.810075351174179
R-squared (XGBoost): -0.3075153744291641


...

,Player_name,Rank,Pos,Points_won,Predicted_points
1022,Rudy Gobert,3.0,C,136.0,211.196228
997,Jaren Jackson Jr.,5.0,PF,99.0,36.325352
628,Mikal Bridges,2.0,SF,202.0,28.701435
1132,Derrick Walton,-1.0,PG,0.0,28.592588
1284,Darius Garland,-1.0,PG,0.0,26.051262
1111,Giannis Antetokounmpo,6.0,PF,58.0,24.176006
644,Evan Mobley,-1.0,PF,0.0,23.615051
933,Al Horford,9,C,3.0,19.899338
1123,Draymond Green,10,PF,2.0,18.541801
758,Wendell Carter Jr.,-1.0,PF,0.0,17.570864


---------------------------------------------------------------------
Selected Features Top  40  Test year :  2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 272.055159090303
R-squared (XGBoost): 0.36064820799121455


,Player_name,Rank,Pos,Points_won,Predicted_points
323,Brook Lopez,2.0,C,309.0,98.229469
206,Nic Claxton,9T,C,7.0,88.755997
137,Evan Mobley,3.0,PF,101.0,84.908257
158,Jrue Holiday,7T,PG,8.0,76.211555
59,Jaren Jackson Jr.,1.0,C,391.0,68.160347
176,Giannis Antetokounmpo,6.0,PF,14.0,62.916126
208,Walker Kessler,-1.0,C,0.0,20.548960
532,Darius Garland,-1.0,PG,0.0,18.560841
190,Desmond Bane,-1.0,SG,0.0,18.464622
497,Joel Embiid,9T,C,7.0,17.487640


---------------------------------------------------------------------


,Rank_1994,Rank_1995,Rank_1996,Rank_1997,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,1,4,3,2,2,5T,1,-1,-1,1,...,1.0,1,1,1,1.0,1.0,1.0,1.0,3.0,2.0
1,4,1,4,-1,1,4,5T,1,1,4,...,4.0,3.0,3,3,8.0,2.0,3.0,5.0,5.0,9T
2,7T,3,-1.0,1,3,1,3T,5T,3T,-1,...,15T,4,2,-1,3,-1.0,2,-1,2.0,3.0
3,2,2,2,4,5T,-1,3T,-1,-1,-1,...,7.0,-1,4.0,5T,6,3.0,-1.0,9.0,-1.0,7T
4,5,6T,5,-1,5T,3,2,3,3T,8,...,11,10T,10,-1.0,-1.0,5.0,11,6.0,-1.0,1.0
5,7T,6T,6,5,5T,2,-1,2,-1,6,...,5,-1,-1,2.0,-1,6T,-1.0,2.0,6.0,6.0
6,10,-1,7,3,9T,-1,8T,-1,-1,-1.0,...,8T,-1,-1.0,-1,-1.0,-1.0,10.0,-1,-1.0,-1.0
7,9,6T,-1,6T,5T,5T,8T,8T,-1,5.0,...,-1.0,2,-1.0,5T,15T,-1,8T,3,9,-1.0
8,-1,5,8T,-1.0,11T,-1,-1,11T,2,13T,...,2.0,6.0,11T,-1,-1.0,-1,-1.0,4.0,10,-1.0
9,6,6T,8T,-1.0,4,7T,-1,1,-1,13T,...,8T,15T,7.0,-1.0,-1,-1,-1,-1.0,-1.0,9T


In [36]:
top = top_50

rank_data_50 = pd.DataFrame()

for year in range(1994,2024):
    print("Selected Features Top ",50," Test year : ", year)
    Test_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] == year]
    Train_Data = all_players_w_add_sorted[all_players_w_add_sorted['Year'] < year]
    
    X_train, y_train = Data_Organizer(Train_Data,top)
    X_test , y_test = Data_Organizer(Test_Data,top)
    
    # Initialize and train a XGBoost Regressor model
    xgb_model = XGBRegressor(n_estimators=200,max_depth = 5, random_state=42, n_jobs=-1)
    xgb_model.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred_xgb = xgb_model.predict(X_test)
    
    # Evaluate the model
    mse_xgb = mean_squared_error(y_test, y_pred_xgb)
    r2_xgb = r2_score(y_test, y_pred_xgb)
    
    # Mean Squared Error (MAE)
    print(f'Mean Squared Error (XGBoost): {mse_xgb}')
    print(f'R-squared (XGBoost): {r2_xgb}')
    
    Test_Data_copy = Test_Data.copy()
    Test_Data_copy.loc[:, 'Predicted_points'] = y_pred_xgb
    
    # Select only the desired columns
    Test_Data_concatenated = Test_Data_copy[['Player_name','Rank','Pos','Points_won', 'Predicted_points']]
    
    # Display the concatenated DataFrame sorted by 'Points_won'
    display(Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10))

    Test_Data_concatenated = Test_Data_concatenated.sort_values(by='Predicted_points', ascending=False).head(10)

    if rank_data_50.empty:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_50 = pd.DataFrame(new_rank_column)
    else:
        new_rank_column = Test_Data_concatenated['Rank'].head(10).rename(f'Rank_{year}')
        rank_data_50 = pd.concat([rank_data_50.reset_index(drop=True), new_rank_column.reset_index(drop=True)], axis=1)

    print("---------------------------------------------------------------------")

display(rank_data_50)

Selected Features Top  50  Test year :  1994
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 7.096991946246465
R-squared (XGBoost): -0.9213517213885805


,Player_name,Rank,Pos,Points_won,Predicted_points
15651,Hakeem Olajuwon,1,C,23.0,72.997948
15477,Scottie Pippen,4,SF,11.0,8.406638
15539,David Robinson,2,C,22.0,7.870159
15583,Mookie Blaylock,7T,PG,3.0,7.276592
15784,Gary Payton,5,PG,9.0,2.130139
15540,Horace Grant,10,PF,1.0,2.130139
15677,Nate McMillan,7T,PG,3.0,2.130139
15739,Patrick Ewing,-1,C,0.0,1.000584
15649,Charles Oakley,9,PF,2.0,1.000237
15743,Dikembe Mutombo,3,C,19.0,1.000237


---------------------------------------------------------------------
Selected Features Top  50  Test year :  1995
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 2.925179890409936
R-squared (XGBoost): 0.5350610873874032


,Player_name,Rank,Pos,Points_won,Predicted_points
15113,David Robinson,4,C,12.0,17.454617
14994,Hakeem Olajuwon,3,C,13.0,17.278801
15026,Dikembe Mutombo,1,C,45.0,17.232346
15151,Gary Payton,6T,PG,2.0,14.055132
15199,Scottie Pippen,2,SF,16.0,11.027144
15404,Alonzo Mourning,6T,C,2.0,8.524605
15121,Dennis Rodman,5,PF,11.0,7.682734
15077,Mookie Blaylock,-1,PG,0.0,7.255709
15258,Derrick McKey,6T,SF,2.0,6.696791
15370,John Starks,-1,SG,0.0,5.585331


---------------------------------------------------------------------
Selected Features Top  50  Test year :  1996
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 9.680395069587112
R-squared (XGBoost): -0.2902312093370889


...

,Player_name,Rank,Pos,Points_won,Predicted_points
1022,Rudy Gobert,3.0,C,136.0,212.763412
1284,Darius Garland,-1.0,PG,0.0,46.276989
1123,Draymond Green,10,PF,2.0,45.569031
933,Al Horford,9,C,3.0,40.440315
942,Fred VanVleet,-1.0,PG,0.0,36.159027
1306,Bam Adebayo,4.0,C,128.0,30.349836
997,Jaren Jackson Jr.,5.0,PF,99.0,30.106382
1111,Giannis Antetokounmpo,6.0,PF,58.0,22.623610
629,Jimmy Butler,-1,SF,0.0,18.740768
678,Chris Paul,-1,PG,0.0,14.712461


---------------------------------------------------------------------
Selected Features Top  50  Test year :  2023
DataFrame does not contain missing values or empty strings/spaces.
DataFrame does not contain missing values or empty strings/spaces.
Mean Squared Error (XGBoost): 335.72646614140547
R-squared (XGBoost): 0.21101544822740603


,Player_name,Rank,Pos,Points_won,Predicted_points
176,Giannis Antetokounmpo,6.0,PF,14.0,65.054092
206,Nic Claxton,9T,C,7.0,62.636051
323,Brook Lopez,2.0,C,309.0,55.956032
392,Jacob Gilyard,-1.0,PG,0.0,47.113819
59,Jaren Jackson Jr.,1.0,C,391.0,46.056057
119,Josh Hart,-1.0,SF,0.0,37.240479
85,P.J. Tucker,-1.0,PF,0.0,32.304375
327,Kevon Looney,-1.0,C,0.0,32.194614
396,Rudy Gobert,-1.0,C,0.0,23.437664
137,Evan Mobley,3.0,PF,101.0,15.961123


---------------------------------------------------------------------


,Rank_1994,Rank_1995,Rank_1996,Rank_1997,Rank_1998,Rank_1999,Rank_2000,Rank_2001,Rank_2002,Rank_2003,...,Rank_2014,Rank_2015,Rank_2016,Rank_2017,Rank_2018,Rank_2019,Rank_2020,Rank_2021,Rank_2022,Rank_2023
0,1,4,3,2,1,5T,1,-1,1,1,...,1.0,1,1,1,1.0,1.0,1.0,1.0,3.0,6.0
1,4,3,4,-1,2,1,5T,5T,3T,-1,...,4.0,3.0,4.0,2.0,8.0,2.0,3.0,5.0,-1.0,9T
2,2,1,-1.0,1,3,4,3T,3,-1,4,...,15T,4,2,3,15T,-1.0,2,9.0,10,2.0
3,7T,6T,6,3,4,-1,2,-1,-1,6,...,5,15T,3,-1.0,-1,5.0,11,3,9,-1.0
4,5,2,2,4,5T,2,3T,1,-1,5.0,...,6,-1,10,5T,-1.0,3.0,8T,-1.0,-1.0,1.0
5,10,6T,5,5,5T,3,-1,8T,2,8,...,7.0,-1,-1.0,-1,-1,-1.0,-1.0,2.0,4.0,-1.0
6,7T,5,7,6T,5T,-1,-1,11T,3T,-1,...,8T,10T,-1,-1.0,3,6T,10.0,-1,5.0,-1.0
7,-1,-1,-1,-1.0,-1,-1,8T,1,-1,-1.0,...,2.0,2,13T,-1.0,15T,-1,-1.0,4.0,6.0,-1.0
8,9,6T,8T,-1,9T,-1,-1,-1,-1,3,...,8T,15T,-1,-1,6,-1,-1.0,-1.0,-1,-1.0
9,3,-1,8T,-1,-1,7T,8T,2,-1,13T,...,13T,5.0,5.0,-1,-1,-1,-1.0,-1,-1,3.0


In [37]:
rank_data_10.to_csv('XGB+XGB_All_rank_data_10.csv', index=False)
rank_data_20.to_csv('XGB+XGB_All_rank_data_20.csv', index=False)
rank_data_30.to_csv('XGB+XGB_All_rank_data_30.csv', index=False)
rank_data_40.to_csv('XGB+XGB_All_rank_data_40.csv', index=False)
rank_data_50.to_csv('XGB+XGB_All_rank_data_50.csv', index=False)